<a href="https://colab.research.google.com/github/Adrianlg01/A1-Introduccion-a-Bases-de-Datos-Santander2021/blob/main/Segmentaci%C3%B3n_de_instancias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[texto del vínculo](https://)## Segmentación de Objetos en Conjunto de Datos Personalizado

**Objetivo:**


### Introducción

El modelo YOLO está diseñado para ser rápido, preciso y fácil de usar, lo que lo convierte en una excelente opción para una amplia gama de tareas de detección de objetos y segmentación de imágenes. Se puede entrenar en grandes conjuntos de datos y es capaz de ejecutarse en una variedad de plataformas de hardware.

**Historia**

* YOLO (You Only Look Once) es un popular modelo de segmentación de imágenes y detección de objetos desarrollado por Joseph Redmon y Ali Farhadi en la Universidad de Washington. La primera versión de YOLO se lanzó en 2015 y ganó popularidad rápidamente debido a su alta velocidad y precisión.

* YOLOv2 se lanzó en 2016 y mejoró el modelo original al incorporar normalización por lotes (*batch normalization*), cuadros de anclaje (*anchor boxes*) y clusters de dimensiones.

* YOLOv3 se lanzó en 2018 y mejoró aún más el rendimiento del modelo mediante el uso de una red espinal (*backbone network*) más eficiente, agregando una pirámide de características y haciendo uso de la pérdida focal (*focal loss*).

* En 2020, se lanzó YOLOv4, que introdujo una serie de innovaciones, como el uso del aumento de datos Mosaico (*Mosaic data augmentation*), un nuevo cabezal de detección sin anclaje (*anchor-free detection head*) y una nueva función de pérdida.

* En 2021, Ultralytics lanzó YOLOv5, que mejoró aún más el rendimiento del modelo y agregó nuevas funciones, como la compatibilidad con la segmentación panóptica (*panoptic segmentation*) y el seguimiento de objetos (*object tracking*).

* 2023, YOLOv8 es la última versión del modelo desarrollado por Ultralytics. Es un modelo de última generación que se basa en el éxito de las versiones anteriores e introduce nuevas características y mejoras para aumentar aún más el rendimiento y la flexibilidad. Una característica clave de YOLOv8 es su extensibilidad pues está diseñado como un marco que admite todas las versiones anteriores de YOLO, lo que facilita cambiar entre diferentes versiones y comparar su rendimiento. Además de su extensibilidad, YOLOv8 incluye una serie de otras innovaciones que incluyen un nuevo *backbone*, un nuevo cabezal de detección sin anclaje y una nueva función de pérdida.

* 2024, con YOLOv11, vemos la culminación de años de innovación ya que se brinda una detección de objetos más rápida, precisa y eficiente que nunca antes.

<img src='https://user-images.githubusercontent.com/27466624/222874205-3873bdac-7135-4ecc-8ab2-ca18b8e13fdf.jpg'>

<img src='https://miro.medium.com/v2/resize:fit:4800/format:webp/1*L8rMuwurmyBH1ixIqcrMSQ.png'>

#### Antes de iniciar

Es importante corroborar que se cuente con acceso a una GPU. Para ello se puede utilizar el comando `nvidia-smi`.

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

### Instalación

YOLOv11 se puede instalar de dos maneras. Utilizando pip o alternativamente, clonando directamente el repositorio.

In [ ]:
# Método de instalación con pip (recomendado)
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
# Método de clonación de repositorio

# %cd {HOME}
# !git clone github.com/ultralytics/ultralytics
# %cd {HOME}/ultralytics
# !pip install -qe ultralytics

# from IPython import display
# display.clear_output()

# import ultralytics
# ultralytics.checks()

In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image

### Inferencia con el modelo preentrenado en el conjunto de datos COCO (common objects in context)

Enlace al repositorio oficial de Ultralytics: https://github.com/ultralytics/ultralytics

#### CLI (interfaz de línea de comandos)

`yolo mode=predict` ejecuta la inferencia de YOLOv11 en una variedad de fuentes, descarga los modelos automáticamente desde la última versión de YOLOv11 y guarda los resultados en `runs/predict`.

In [ ]:
%cd {HOME}
!yolo task=segment mode=predict model=yolo11l-seg.pt conf=0.25 source='https://pixnio.com/free-images/2017/04/10/2017-04-10-07-10-35.jpg' save=True

In [ ]:
%%time
%cd {HOME}
Image(filename='runs/segment/predict/2017-04-10-07-10-35.jpg', height=600)

#### SDK (kit de desarrollo de software)

In [ ]:
model = YOLO(f'{HOME}/yolo11s-seg.pt')
results = model.predict(source='https://pixnio.com/free-images/2017/04/10/2017-04-10-07-10-35.jpg', conf=0.25)

In [ ]:
results[0].boxes.xyxy

In [ ]:
results[0].boxes.conf

In [ ]:
results[0].boxes.cls

In [ ]:
#results[0].masks

## Conjuntos de datos

Se recomienda ampliamente el uso de Roboflow. Se pueden encontrar múltiples conjuntos de datos para practicar en la página de Roboflow:

https://universe.roboflow.com/

## Preparar un conjunto de datos personalizado

Crear un conjunto de datos personalizado puede ser un proceso lento. Puede tomar docenas o incluso cientos de horas recopilar imágenes, etiquetarlas y exportarlas en el formato adecuado. Afortunadamente, Roboflow hace que este proceso sea lo más sencillo y rápido posible.

### Paso 1: Crear un proyecto

Antes de comenzar, se debe crear una [cuenta](https://app.roboflow.com/login) con Roboflow. Una vez hecho esto, se puede crear un nuevo proyecto dentro de la [interfaz](https://app.roboflow.com/) de Roboflow. Hay que tener en cuenta que se debe elegir el tipo de proyecto adecuado. A continuación un ejemplo:

<div align="center">
  <img
    width="640"
    src="https://ik.imagekit.io/roboflow/preparing-custom-dataset-example/creating-project.gif?ik-sdk-version=javascript-1.4.3&updatedAt=1672929799852"
  >
</div>

### Paso 2: Cargar imágenes

A continuación se agregan los datos al proyecto recién creado. Esto se puede realizar a través de API o a través de la [plataforma](https://docs.roboflow.com/adding-data/object-detection).

Si se arrastra y suelta un directorio con un conjunto de datos en un formato compatible, la interfaz de Roboflow cargará automáticamente las imágenes y las etiquetas juntas.

<div align="center">
  <img
    width="640"
    src="https://ik.imagekit.io/roboflow/preparing-custom-dataset-example/uploading-images.gif?ik-sdk-version=javascript-1.4.3&updatedAt=1672929808290"
  >
</div>

### Paso 3: Etiquetado

Si solo se cuenta con imágenes (no etiquetadas), se puede etiquetar directamente en el apartado de [anotaciones](https://docs.roboflow.com/annotate).

<div align="center">
  <img
    width="640"
    src="https://user-images.githubusercontent.com/26109316/210901980-04861efd-dfc0-4a01-9373-13a36b5e1df4.gif"
  >
</div>

### Paso 4: Generar una nueva versión del conjunto de datos

Ahora que se han añadido las imágenes y sus etiquetas, se puede generar una nueva versión del conjunto de datos. Al generar una versión, se puede optar por agregar preprocesamiento y aumentación de datos. Este paso es opcional, sin embargo, puede permitir mejorar significativamente la robustez del modelo.

<div align="center">
  <img
    width="640"
    src="https://media.roboflow.com/preparing-custom-dataset-example/generate-new-version.gif?ik-sdk-version=javascript-1.4.3&updatedAt=1673003597834"
  >
</div>

### Paso 5: Exportar el conjunto de datos

Una vez que se genera una versión del conjunto de datos, éste puede ser cargado directamente en Google Colab para facilitar el entrenamiento, haciendo clic en "Exportar" y seleccionando el formato de conjunto de datos adecuado para el modelo.

<div align="center">
  <img
    width="640"
    src="https://ik.imagekit.io/roboflow/preparing-custom-dataset-example/export.gif?ik-sdk-version=javascript-1.4.3&updatedAt=1672943313709"
  >
</div>

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("").project("")
version = project.version(1)
dataset = version.download("yolov11")

### Entrenamiento personalizado

In [ ]:
dataset.location

In [ ]:
%%time
%cd {HOME}
!yolo task=segment mode=train model=yolo11s-seg.pt data={dataset.location}/data.yaml epochs=25 imgsz=640 plots=True

### Evaluación

In [ ]:
!ls {HOME}/runs/segment/train/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train/results.png', width=800)

### Validaciones

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train/val_batch0_pred.jpg', width=600)

In [ ]:
%cd {HOME}
!yolo task=segment mode=val model={HOME}/runs/segment/train/weights/best.pt data={dataset.location}/data.yaml

### Inferencia

In [ ]:
%%time
%cd {HOME}
!yolo task=segment mode=predict model={HOME}/runs/segment/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/segment/predict2/*.jpg')[:10]:
      display(Image(filename=image_path, height=600))
      print("\n")

**Referencias**

* https://paperswithcode.com/task/instance-segmentation
* https://github.com/roboflow/notebooks
* https://universe.roboflow.com/
* https://docs.ultralytics.com/
* https://arxiv.org/abs/1506.02640